<center><h1>万字逐行解析与实现Transformer，并进行德译英实战</h1> </center>

# 本文内容

本文代码来源于[该链接](https://github.com/harvardnlp/annotated-transformer/)，该作者对Transformer进行了详细解释，并进行了德译英实战，且增加了许多注解。但对于像我这样的新手来讲还是有些难度，所以我在该作者代码与注释的基础上，进一步增加注释，将英文注释改为中文注释，且去掉了一些与理解Transformer无关的代码（例如并行计算等），方便大家理解Transformer。

该教程需要读者对Transformer有一些了解。

本篇中可能有些注释比较难懂，请不要慌张，可以先打个标记，因为大部分都会在后面对其进行讲解。

如果本篇有什么错误的地方或写的不清楚的地方，欢迎在评论区或者issue模块提出来，我会进行改正或进一步解释。

本文主要内容有：

1. Transformer各模块的逐行代码实现
2. Transformer逐行代码的详细注释
3. Transformer的训练和推理
4. 利用Transformer进行德译中实战

你可以通过[Google Colab](https://drive.google.com/file/d/172BrYvcJpQc9eIciZpyjc7RpqPDWs6pe/view?usp=sharing)直接运行以下代码

# 环境准备

本教程使用到的类库版本如下：

```
pandas==1.3.5
torch==1.11.0+cu113
torchdata==0.3.0
torchtext==0.12
spacy==3.2
altair==4.1
jupytext==1.13
flake8
black
GPUtil
```

一定要按照该版本进行安装，尤其是torchtext与torchdata，否则后面会报错

In [1]:
!pip install pandas==1.3.5
!pip install torch==1.11.0+cu113
!pip install torchdata==0.3.0
!pip install torchtext==0.12
!pip install spacy==3.2
!pip install altair==4.1
!pip install jupytext==1.13
!pip install flake8
!pip install black
!pip install GPUtil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.11.0+cu113 (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0)
ERROR: No matching distribution found for torch==1.11.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 3.1 MB/s 
     |████████████████████████████████| 139 kB 12.9 MB/s 
     |████████████████████████████████| 750.6 MB 10 kB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled url

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 727 kB 6.9 MB/s 
  Attempting uninstall: altair
    Found existing installation: altair 4.2.0
    Uninstalling altair-4.2.0:
      Successfully uninstalled altair-4.2.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 295 kB 7.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 271 kB/s 
     |████████████████████████████████| 66 kB 4.5 MB/s 
     |████████████████████████████████| 41 kB 434 kB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.12.0
    Uninstalling importlib-metadata-4.12.0:
      Succe

除了上面的库外，还需要用到spacy进行分词。关于spacy可参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/125469155)。在安装完spacy后，需要安装德语和英语的库，命令如下：

In [ ]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.1 MB 7.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.9 MB 8.2 MB/s 


接下来导入需要用到的类库：

In [ ]:
import os
from os.path import exists
import torch
import torch.nn as nn
# Pad: 用于对句子进行长度填充，官方地址为：https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html
from torch.nn.functional import log_softmax, pad
import math
# copy: 用于对模型进行深拷贝
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
# 和matplotlib.pypolt类似，用于绘制统计图，但功能功能强大
# 可以绘制可交互的统计图。官网地址为：https://altair-viz.github.io/getting_started/overview.html
import altair as alt
# 用于将 iterable 风格的 dataset 转为 map风格的 dataset，详情可参考：https://blog.csdn.net/zhaohongfei_358/article/details/122742656
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
# 用于构建词典
from torchtext.vocab import build_vocab_from_iterator
# datasets：用于加载Multi30k数据集
import torchtext.datasets as datasets
# spacy: 一个易用的分词工具，详情可参考：https://blog.csdn.net/zhaohongfei_358/article/details/125469155
import spacy
# GPU工具类，本文中用于显示GPU使用情况
import GPUtil
# 用于忽略警告日志
import warnings

# 设置忽略警告
warnings.filterwarnings("ignore")

In [ ]:
# 用于验证时，不进行参数更新。
class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None

# 用于验证时，不进行学习率调整。
class DummyScheduler:
    def step(self):
        None

# 背景介绍

# Part 1: 模型架构

# Model Architecture

大部分序列模型(neural sequence transduction models)都是encoder-decoder结构。encoder负责将一个符号表示的输入序列 $(x_1, ..., x_n)$ 映射为一个连续表示的序列$\mathbf{z} = (z_1, ...,
z_n)$。然后将$\mathbf{z}$作为Decoder的其中一个输入，decoder会一次一个的产生字符输出序列（output sequence of symbols）$(y_1,...,y_m)$。在每个时刻，模型都是自回归的（auto-regressive），也就是上一个时刻的产生的字符，作为写一个时刻额外的输入。

而Transformer也是这样的encoder-decoder结构，一个标准的EncoderDecoder模型如下：

In [ ]:
class EncoderDecoder(nn.Module):
    """
    一个标准的EncoderDecoder模型。在本教程中，这么类就是Transformer
    """

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        """
        encoder: Encoder类对象。Transformer的Encoder
        decoder： Decoder类对象。 Transformer的Decoder
        src_embed: Embeddings类对象。 Transformer的Embedding和Position Encoding
                   负责对输入inputs进行Embedding和位置编码
        tgt_embed: Embeddings类对象。 Transformer的Embedding和Position Encoding
                   负责对“输入output”进行Embedding和位置编码
        generator: Generator类对象，负责对Decoder的输出做最后的预测（Linear+Softmax）
        """
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        src: 未进行word embedding的句子，例如`[[ 0, 5, 4, 6, 1, 2, 2 ]]`
             上例shape为(1, 7)，即batch size为1，句子词数为7。其中0为bos，
             1为eos, 2为pad

        tgt: 未进行word embedding的目标句子，例如`[[ 0, 7, 6, 8, 1, 2, 2 ]]`

        src_mask: Attention层要用的mask，对于source来说，就是要盖住非句子的部分，
                  例如`[[True,True,True,True,True,False,False]]`。相当于对上面
                  `[[ 0, 5, 4, 6, 1, 2, 2 ]]`中最后的`2,2`进行掩盖。

        tgt_mask: Decoder的Mask Attention层要用的。该shape为(N, L, L)，其中N为
                  batch size, L为target的句子长度。例如(1, 7, 7)，对于上面的例子，
                  值为：
                  [True,False,False,False,False,False,False], # 从该行开始，每次多一个True
                  [True,True,False,False,False,False,False],
                  [True,True,True,False,False,False,False],
                  [True,True,True,True,False,False,False],
                  [True,True,True,True,True,False,False], # 由于该句一共5个词，所以从该行开始一直都只是5个True
                  [True,True,True,True,True,False,False],
                  [True,True,True,True,True,False,False],
        """

        # 注意，这里的返回是Decoder的输出，并不是Generator的输出，因为在EncoderDecoder
        # 的forward中并没有使用generator。generator的调用是放在模型外面的。
        # 这么做的原因可能是因为Generator不算是Transformer的一部分，它只能算是后续处理
        # 分开来的话也比较方便做迁移学习。另一方面，推理时只会使用输出的最后一个tensor送给
        # generator，而训练时会将所有输出都送给generator。
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        """
        该encode做三件事情：
        1. 对src进行word embedding
        2. 将word embedding与position encoding相加
        3. 通过Transformer的多个encoder层计算结果，输出结果称为memory
        """
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        """
        memory: Transformer Encoder的输出

        decoder和encoder执行过程基本一样：
        1. 对src进行word embedding
        2. 将word embedding与position encoding相加
        3. 通过Transformer的多个decoder层计算结果

        当完成decoder的计算后，接下来可以使用self.generator（nn.Linear+Softmax）来进行最后的预测。
        """
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

上述的代码中包含几个重要的模块，与Transformer的对应关系如下图：

<img src="./images/transfomer_component.png" width="400">


我们根据上图来对Transformer进行拆解：

1. **模型的输入**：Transformer模型的输入包含两部分：① Inputs ② Outputs。这里可能就会问了Outputs不是输出吗？怎么成输入了？且看后面解释
    1.1 **Inputs**: Inputs可以理解为原始文本，例如我们可以将“I love you”根据字典对应的index转换为数字后（例如`[5, 4, 6]`），作为inputs传给Transformer的Encoder层。
    1.2 **Outputs**: Outputs是上一次Decoder的输出。例如我们在做机器翻译的时候，Encoder接收的是一个完整的句子（假设为I love you），然后输出一个`memory`(在RNN里叫Hidden State)，但是Decoder并不是一下子就能输出“我爱你”，它和RNN一样，要一个字一个字输出。而Outputs就是上次的输出。（这里看不懂没关系，后面会完整的举个例子）
2. **Embedding**: 在Inputs和Outputs的上面有两个Embedding，`Input Embedding`和`Output Embedding`，它们的作用是对字符进行编码。如果你不是特别了解，可以参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/122809709)。通常Embedding层使用`nn.Embedding`就可以了。
3. **Positional Encoding**: Transformer和RNN相比，一个重要的提升就是Transformer可以并行计算，其实就是Transformer的Encoder可以一次性接收整个句子，而RNN需要一个词一个词接收。但一次性接收整个句子就会淡化句子中词与词的位置关系，例如 “我爱你”和“爱你我” 可能对网络来说没什么不一样的。 为了解决这个问题，Transformer在Embedding后给每个词增加了位置成分。例如：可以让 `I` 向量的每个值都加0.1，`Love`加0.2，`you`加0.3。 当然，Transformer使用的机制并不是像这个例子那么简单，但道理是一样的。
4. **Encoder**：Encoder就是将Inputs进行编码，在效果上和RNN的Encoder应该也没啥不一样的，但因为增加了Attention机制，Performance有了大幅提升。注意这里的 $Nx$ 指的是有N个这样的Encoder层叠加。
5. **Decoder**: 和RNN类似，接收“Encoder的输出（称为Memory）和上一次Decoder的输出”作为输入，然后进行一系列运算得到输出。
6. **Generator**：Decoder的输出并不能直接拿来用，还要在经过一个线性层才能得到你想要的结果。有点类似CNN中，卷积层之后需要再接一个线性层做分类。注意：在推理时，generator使用的并不是decoder的所有输出，而是最后一个`out[:, -1]`，而在训练时，则是使用Decoder的全部输出。
7. **Output Probabilities**: 这个就是线性层后经过Softmax的概率分布。最大的值对应的index，然后再去字典中查询，就知道预测的词是什么了。

有了上述对Transformer各层的解释，我们来看一下Transformer实际是怎么使用的。这里还使用上述翻译的例子：

首先，假设我们有两个字典：

- `[0(<bos>), 1(<eos>), 2(<pad>), 3(<unk>), 4(Love), 5(I), 6(You), ..., 100(other)]`
- `[0(<bos>), 1(<eos>), 2(<pad>), 3(<unk>), 4(爱), 5(我), 6(你), ..., 100(其他)]`



则我们翻译的第一步如下图：

<img src="./images/encoder.png" width="500">

上图为Encoder的执行过程：

1. 首先我们根据字典将`I Love you` 转变为了向量 `[[ 0, 5, 4, 6, 1, 2, 2 ]]`，这里的Shape为(1,7)，1是batch size, 7是句子长度。而向量中的0表示开始(`<bos>`)，1表示结束(`<eos>`)，2表示填充(`<pad>`)。这里我假设句子长度固定为7，所以需要填充两个字符。
2. 当我们将文字转换为向量后，就会经过Embedding层对向量进行编码，我们这里将一个字符编码成128维的向量，而这个值也对应模型中的参数`d_model`(dimension of model)。而对向量编码一般使用参数可学习的`nn.Embedding`来实现。具体可参考[该篇文章](https://blog.csdn.net/zhaohongfei_358/article/details/122809709)
3. 接下来我们使用Position Encoding为我们的Embedding增加位置成分。这里我假设位置1就是加0.1，位置2加0.2，依次类推
4. 在完成位置编码后，就会正式进入Encoder层，我们有$X$个Encoder层，所以一层一层计算，最终得到Encoder的输出Memory，这个Memory的Shape与输入是一致的。

接下来我们来看Decoder的执行过程，如图所示：

<img src="./images/decoder.png" width="500">

Decoder的执行过程与Encoder类似，区别主要有以下：

1. **输入为目标句子**：Encoder的输入是原句子（即要被翻译的句子），而Decoder的输入是目标句子（即翻译后的句子）。在推理时，Decoder是一遍一遍的执行，每次的输入都是之前的所有输出，例如，第一次输入为`<bos>`，输出为`我`，则第二次输入为`<bos> 我`, 输出为`爱`，依次类推。而在训练时，是一次将目标句子全部送给Decoder，通过掩码（mask）的方式来得到和推理一个一个送同样的结果。
2. 第二步与Encoder一致
3. Decoder层的输入除了有上一层的输出外，还需要Encoder的输出Memory。在Encoder层的Attention层，query、key和value都是输入x（即前一层的输出）。而在Decoder的Attention层，query是x，但key和value都是memory.
4. 在Decoder层输出结果后，需要经过Generator线性层进行最后的预测，其会输出一个概率分布根据这个概率分布找出最大值对应的index，然后查词典就可以了。这个就是个标准的分类网络。需要注意的一点是，在推理时，只需要拿Decoder输出的最后一个tensor送给Generator，得到一个词的概率分布，就像上图画的那样。而在训练时，需要将Decoder的所有输出送给Generator，然后得到每个词的预测结果。

在Decoder输出结果后，需要将结果送给Generator(Linear+Softmax)来预测下一个词(token)。

In [ ]:
class Generator(nn.Module):
    """
    Decoder的输出会送到Generator中做最后的预测。
    """

    def __init__(self, d_model, vocab):
        """
        d_model: dimension of model. 这个值其实就是word embedding的维度。
                 例如，你把一个词编码成512维的向量，那么d_model就是512
        vocab: 词典的大小。
        """
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        """
        x为Decoder的输出，例如x.shape为(1, 7, 128)，其中1为batch size, 7为句子长度，128为词向量的维度

        这里使用的是log_softmax而非softmax，效果应该是一样的。
        据说log_softmax能够解决函数overflow和underflow，加快运算速度，提高数据稳定性。
        可以参考：https://www.zhihu.com/question/358069078/answer/912691444
        该作者说可以把softmax都可以尝试换成log_softmax
        """
        return log_softmax(self.proj(x), dim=-1)

## Encoder and Decoder 的堆叠

### Encoder

Encoder是由多个相同的EncoderLayer堆叠而成的。原图中只画了一个EncoderLayer，然后旁边写了个 $N_x$ 。这个Nx就表示Encoder是由几个EncoderLayer堆叠而成。默认$N=6$

In [ ]:
def clones(module, N):
    """
    该方法负责产生多个相同的层。
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    """
    Encoder的核心部分就是多个相同的EncoderLayer堆叠而成。
    """

    def __init__(self, layer, N):
        """
        初始化传入两个参数：
        layer: 要堆叠的层，对应下面的EncoderLayer类
        N: 堆叠多少次
        """
        super(Encoder, self).__init__()
        # 将Layer克隆N份
        self.layers = clones(layer, N)
        # LayerNorm层就是BatchNorm。也就是对应Transformer中
        # “Add & Norm”中的“Norm”部分
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """
        x: 进行过Embedding和位置编码后的输入inputs。Shape为(batch_size, 词数，词向量维度)
           例如(1, 7, 128)，batch_size为1，7个词，每个词128维
        mask: src_mask，请参考EncoderDecoder.forward中的src_mask注释
        """

        # 一层一层的执行，前一个EncoderLayer的输出作为下一层的输入
        for layer in self.layers:
            x = layer(x, mask)

        # 你可能会有疑问，为什么这里会有一个self.norm(x)，
        # 这个疑问会在后面的`SublayerConnection`中给出解释
        return self.norm(x)

下面是LayerNorm的代码实现，这个应该算是`torch.nn.BatchNorm2d`一个简单的实现：

In [ ]:
class LayerNorm(nn.Module):
    """
    Norm层，其实该层的作用就是BatchNorm。与`torch.nn.BatchNorm2d`的作用一致。
    torch.nn.BatchNorm2d的官方文档地址：https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

    该LayerNorm就对应原图中 “Add & Norm”中“Norm”的部分
    """

    def __init__(self, features, eps=1e-6):
        """
        features: int类型，含义为特征数。也就是一个词向量的维度，例如128。该值一般和d_model一致。
        """
        super(LayerNorm, self).__init__()
        """
        这两个参数是BatchNorm的参数，a_2对应gamma(γ), b_2对应beta(β)。
        而nn.Parameter的作用就是将这个两个参数作为模型参数，之后要进行梯度下降。
        """
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        # epsilon，一个很小的数，防止分母为0
        self.eps = eps

    def forward(self, x):
        """
        x： 为Attention层或者Feed Forward层的输出。Shape和Encoder的输入一样。（其实整个过程中，x的shape都不会发生改变）。
            例如，x的shape为(1, 7, 128)，即batch_size为1，7个单词，每个单词是128维度的向量。
        """

        # 按最后一个维度求均值。mean的shape为 (1, 7, 1)
        mean = x.mean(-1, keepdim=True)
        # 按最后一个维度求方差。std的shape为 (1, 7, 1)
        std = x.std(-1, keepdim=True)
        # 进行归一化，详情可查阅BatchNorm相关资料。
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

接下来是“Add & Norm”部分，在该代码中称为`SublayerConnection`，因为“Add & Norm”就是两个子层之间的残差连接。关于残差连接，可以参考ResNet网络。

In [ ]:
class SublayerConnection(nn.Module):
    """
    在一个Norm层后跟一个残差连接。
    注意，为了代码简洁，Norm层是在最前面，而不是在最后面
    """

    def __init__(self, size, dropout):
        """
        这里的size就是d_model。也就是词向量的维度
        """
        super(SublayerConnection, self).__init__()
        # BatchNorm层
        self.norm = LayerNorm(size)
        # BatchNorm的dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """
        x：本层的输入，前一层的输出。
        sublayer: 这个Sublayer就是Attention层或Feed ForWard层。
        """
        return x + self.dropout(sublayer(self.norm(x)))

在上面的代码中，有这么一句注释：**Norm层是在最前面，而不是在最后面**。这句话怎么理解呢？

在Transformer原图中，Encoder架构是这样：`Attention` -> `Add & Norm` -> `Feed Forward` -> `Add & Norm`。

所以我们会下意识地认为代码也是按照这个顺序实现的，但实际代码为了简单，是按照下图这个顺序实现的：

<img src="./images/encoder2.png" width="500">


从上图可以看到，Norm是在是放在最前面的，而不是最后面，所以在`SublayerConnection`中的代码是：

```
return x + self.dropout(sublayer(self.norm(x)))
```

而不是

```
return self.norm(x + self.dropout(sublayer(x)))
```

同时我们也能发现另外两点：

1. 在向量经过Position Encoding后到进入Encoder前是有一个Norm动作的。
2. 最后一个EncoderLayer层Add后并没有Norm，所以要补一个，这也就是为什么在Encoder类的返回是`return self.norm(x)`而不是直接`return x`。

Encoder的每一层都有两个子层(sub-layers)，第一个是多头注意力机制（multi-head
self-attention mechanism），第二个是一个简单的全连接前馈神经网络（Feed Forward）。

In [ ]:
class EncoderLayer(nn.Module):
    "Encoder由一个Attention层和一个前馈网络组成"

    def __init__(self, size, self_attn, feed_forward, dropout):
        """
        size: 就是d_model，也是词向量的维度。
        self_attn: MultiHead Self-Attention模型
        feed_forward: 前馈神经网络
        """
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        # 克隆两个SublayerConnection，第一个给Attention用，第二个给Feed Forward用
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        """
        该方法对应原图中Encoder的部分
        """
        # 首先执行Attention层的计算
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # 接着执行FeedForward层的计算
        return self.sublayer[1](x, self.feed_forward)

### Decoder

Decoder和Encoder类似，是由 $N$ 个相同的DecoderLayer层堆积而成的。默认 $N=6$


In [ ]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."

    def __init__(self, layer, N):
        """
        layer: 要堆叠的层，对应下面的DecoderLayer类
        N: 堆叠多少次
        """
        super(Decoder, self).__init__()
        # 克隆出N个DecoderLayer
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        """
        x: 进行过Embedding和位置编码后的“输入outputs”。Shape为(batch_size, 词数，词向量维度)
           例如(1, 7, 128)，batch_size为1，7个词，每个词128维
           在预测时，x的词数会不断变化，x的shape第一次为(1, 1, 128)，第二次为(1, 2, 128)，以此类推。

        其他参数注释请参考上面的EncoderDecoder类
        """
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

除了Encoder的两个sublayer外，decoder还在中间插入了第三个子层（就是Decoder中Feed Forward和Masked Multi-head Attention中间的那个Multi-Head Attention），该子层也是一个Attention层，只不过这次Attention是针对Encoder的输出Memory和Decoder的第一个Attention的输出。与Encoder类似，每个子层也都做了“Add & Norm”。

In [ ]:
class DecoderLayer(nn.Module):
    """
    Decoder由三部分组成：
    1. self_attn: 对应图中的Masked Multi-head Attention，负责对Outputs做Attention。
    2. src_attn：对应图中Decoder中间的Multi-head Attention，负责对前面Attention的输出
                 和Encoder的输出做Attention
    3. feed_forward：对应Decoder的Feed Forward
    """

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        """
        size: d_model，也就是词向量的维度。
        """
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        # 因为Decoder有三个sublayer，所以拷贝3份。
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "对应原图中Decoder的部分"
        m = memory
        # DecoderLayer的第一个Attention，对应的Mask MultiHead Attention
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # DecoderLayer的第二个Attention（中间那个），attention的key和value使用的是memory
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        # DecoderLayer最后的Feed Forward层。
        return self.sublayer[2](x, self.feed_forward)

Transformer的Attention层还有一个mask机制，主要是为了防止词向量注意一些不该注意的内容，例如：对于Encoder来说，不应该注意`<pad>`的部分，因为这不属于句子成分。对于Decoder来说，前面的词不应该注意后面的词，同时，也不能注意`<pad>`的部分。

关于注意力机制和Mask，可参考[该篇文章](https://blog.csdn.net/zhaohongfei_358/article/details/122861751)

In [ ]:
def subsequent_mask(size):
    """
    生成一个大小为 size x size 的方阵，该方阵对角线与左下全为True，右上全为False。
    例如，size为5时，则结果为：
    [[[ True, False, False, False, False],
      [ True,  True, False, False, False],
      [ True,  True,  True, False, False],
      [ True,  True,  True,  True, False],
      [ True,  True,  True,  True,  True]]]

    该方法在会在构建tgt的mask时使用。
    """

    # 生成一个Shape为(1, size, size)的矩阵，这里要在
    # 前面加个1是为了和tgt的tensor维度保持一致，
    # 因为tgt最前面是batch_size
    attn_shape = (1, size, size)

    """
    生成一个左上角全为1的方阵，例如，当attn_shape为5时，结果为：
       [[[0., 1., 1., 1., 1.],
         [0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]]
    """
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )

    # 将 0 全部变为True, 1变为False
    return subsequent_mask == 0

### Attention


Attention机制比较复杂，这里只是做简单的说明，如果感兴趣，可以参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/122861751)


Self-Attention如果只看公式，其实也简单，就是执行下面这个公式即可：

$$
   \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$

如果用图表示该公式的话，则为：

![](./images/ModalNet-19.png)

大部分对上述的疑问可能就是Q, K, V是怎么来的，$d_k$ 又是什么。其实是这样，在Self-Attention层会训练三个矩阵：$W^q, W^k和W^v$，然后用这三个矩阵乘以输入即可，即：

$$
Q = W^q I \\
K = W^k I \\
V = W^v I \\
$$

这里的I就是EncoderLayer层的输入。而 $d_k$ 就是`d_model`的值，例如128。（如果是MultiHead Attention，则`d_k = d_model / h`，h是head数量。所以这也是为什么d_model必须要能整除head数）

通常我们定义这三个$W$矩阵是通过`nn.Linear`。如果你对`nn.Linear`的本质不太清楚，可以参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/122797190)

> 这里的三个$W$矩阵通常都是方阵，而Q,K,V的shape与输入I的shape一致。

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    """
    计算Attention的结果。
    这里其实传入的是Q,K,V，而Q,K,V的计算是放在模型中的，请参考后续的MultiHeadedAttention类。

    这里的Q,K,V有两种Shape，如果是Self-Attention，Shape为(batch, 词数, d_model)，
                           例如(1, 7, 128)，即batch_size为1，一句7个单词，每个单词128维

                           但如果是Multi-Head Attention，则Shape为(batch, head数, 词数，d_model/head数)，
                           例如(1, 8, 7, 16)，即Batch_size为1，8个head，一句7个单词，128/8=16。
                           这样其实也能看出来，所谓的MultiHead其实就是将128拆开了。

                           在Transformer中，由于使用的是MultiHead Attention，所以Q,K,V的Shape只会是第二种。

    mask： mask有两种，一种是src_mask，另一种是tgt_mask。
    """

    # 获取d_model的值。之所以这样可以获取，是因为query和输入的shape相同，
    # 若为Self-Attention，则最后一维都是词向量的维度，也就是d_model的值。
    # 若为MultiHead Attention，则最后一维是 d_model / h，h为head数
    d_k = query.size(-1)
    # 执行QK^T / √d_k
    # 这里scores是一个方阵, shape为(batch_size, head数，词数，词数)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    """
    mask有两种：
    第一种为： src_mask。Shape为(batch_size, 1, 1, 词数)。例如：(1, 1, 1, 10)。
              [[[1., 1., 1., 1., 1., 1., 1., 0., 0., 0.]]]，表示第一个句子的
              前7个是单词（包括bos、eos和unk），后面3个是填充。而mask就是要对
              后面三个进行mask。
              例如，mask前，score的shape为(1, 2, 10, 10)，2个head，10个词
              score在mask前为：
              [[[[x1, x2, ..., x7, x8, x9, x10],
                 ...
                 [x1, x2, ..., x7, x8, x9, x10]]
                 [x1, x2, ..., x7, x8, x9, x10],
                 ...
                 [x1, x2, ..., x7, x8, x9, x10]]]]
             score在mask后为：
              [[[[x1, x2, ..., x7, -1e9, -1e9, -1e9],
                 ...
                 [x1, x2, ..., x7, -1e9, -1e9, -1e9]]
                 [x1, x2, ..., x7, -1e9, -1e9, -1e9],
                 ...
                 [x1, x2, ..., x7, -1e9, -1e9, -1e9]]]]

    第二种为：tgt_mask。Shape为(batch_size, 1, 词数, 词数)。例如：(1, 1, 10, 10)。例如：
             [[[1, 0, 0, 0, 0],
               [1, 1, 0, 0, 0],
               [1, 1, 1, 0, 0],
               [1, 1, 1, 1, 0],
               [1, 1, 1, 1, 1]]
             即对角线的右上面都盖住。具体为什么这样可以参考链接：https://blog.csdn.net/zhaohongfei_358/article/details/125858248
             score在mask前为：
             [[[[x1, x2, ..., x7, x8, x9, x10],
                 ...
                 [x1, x2, ..., x7, x8, x9, x10]]
                 [x1, x2, ..., x7, x8, x9, x10],
                 ...
                 [x1, x2, ..., x7, x8, x9, x10]]]]
            score在mask后为：
             [[[[x1, 1e9, ..., 1e9, -1e9, -1e9, -1e9],
                 ...
                [x1, x2, ..., 1e9, -1e9, -1e9, -1e9]]
                [x1, x2, ..., 1e9, -1e9, -1e9, -1e9],
                ...
                [x1, x2, ..., x7, x8, x9, x10]]]]

    两者比较：src_mask的shape为(batch_size, 1, 1, 词数)，而tgt_mask的shape为(batch_size, 1, 词数, 词数)
             这是因为src要对后面非句子的词整个mask，所以只需要在最后一维做就行了。
             而tgt需要斜着进行mask，所以需要一个方阵来进行。
    """
    if mask is not None:
        # 这里的填充值并不是0，而是-1e9，原因是后续还要进行softmax，
        # 而softmax会让负无穷变为0。
        scores = scores.masked_fill(mask == 0, -1e9)

    # 执行公式中的Softmax
    # 这里的p_attn是一个方阵
    # 若是Self Attention，则shape为(batch, 词数, 次数)，例如(1, 7, 7)
    # 若是MultiHead Attention，则shape为(batch, head数, 词数，词数)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)

    # 最后再乘以 V。
    # 对于Self Attention来说，结果Shape为(batch, 词数, d_model)，这也就是最终的结果了。
    # 但对于MultiHead Attention来说，结果Shape为(batch, head数, 词数，d_model/head数)
    # 而这不是最终结果，后续还要将head合并，变为(batch, 词数, d_model)。不过这是MultiHeadAttention
    # 该做的事情。
    return torch.matmul(p_attn, value), p_attn

Multi Attention建议参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/122861751)。

Multi Attention简单点说，就是将Q, K, V的最后一个维度进行拆分，拆成了多个head，然后再带入attention函数进行计算，最终再将多个head融合到一起。

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        """
        h: head的数量
        """
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        # 定义W^q, W^k, W^v和W^o矩阵。
        # 如果你不知道为什么用nn.Linear定义矩阵，可以参考该文章：
        # https://blog.csdn.net/zhaohongfei_358/article/details/122797190
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        """
        query, key, value: 这里的这三个变量名容易对人产生误导，
        它指的并不是公式中经过W^q, W^k, W^v计算后的Q, K, V，
        而是要跟 W^q, W^k, W^v 计算的输入。
        对于Encoder的Attention层来说，这里的query,key,value全都是输入x，
        从EncoderLayer类中的`self.self_attn(x, x, x, mask)`这段代码也
        可以看出来。
        对于Decoder最前面那个Mask Attention，query,key,value也全都是输入x。
        但对于Decoder中间的那个Attention就不一样了，它是将Encoder的输出memory
        作为key和value，将输入x作为query。通过DecoderLayer中的代码
        `self.src_attn(x, m, m, src_mask)`可看出来。
        """

        if mask is not None:
            # 原本mask和query的Tensor维度都是3，即len(mask.size())为3
            # 但是由于是多头注意力机制，所以query的维度会变成4，也
            # 就是在第2个维度插入了head，变为了(batch, head数, 词数，d_model/head数)。
            # mask为了和query的维度保持一致，所以也要将自己扩展成4维。
            mask = mask.unsqueeze(1)

        # 获取Batch Size
        nbatches = query.size(0)

        """
        1. 求出Q, K, V，这里是求MultiHead的Q,K,V，所以Shape为(batch, head数, 词数，d_model/head数)
            1.1 首先，通过定义的W^q,W^k,W^v求出SelfAttention的Q,K,V，此时Q,K,V的Shape为(batch, 词数, d_model)
                对应代码为 `linear(x)`
            1.2 分成多头，即将Shape由(batch, 词数, d_model)变为(batch, 词数, head数，d_model/head数)。
                对应代码为 `view(nbatches, -1, self.h, self.d_k)`
            1.3 最终交换“词数”和“head数”这两个维度，将head数放在前面，最终shape变为(batch, head数, 词数，d_model/head数)。
                对应代码为 `transpose(1, 2)`
        """
        query, key, value = [
            linear(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for linear, x in zip(self.linears, (query, key, value))
        ]

        """
        2. 求出Q,K,V后，通过attention函数计算出Attention结果，
           这里x的shape为(batch, head数, 词数，d_model/head数)
           self.attn的shape为(batch, head数, 词数，词数)
        """
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        """
        3. 将多个head再合并起来，即将x的shape由(batch, head数, 词数，d_model/head数)
           再变为 (batch, 词数，d_model)
           3.1 首先，交换“head数”和“词数”，这两个维度，结果为(batch, 词数, head数, d_model/head数)
               对应代码为：`x.transpose(1, 2).contiguous()`
           3.2 然后将“head数”和“d_model/head数”这两个维度合并，结果为(batch, 词数，d_model)
        """
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )

        del query
        del key
        del value

        # 最终通过W^o矩阵再执行一次线性变换，得到最终结果。
        return self.linears[-1](x)

## Position-wise 前馈神经网络

除了Attention子层外，Encoder和Decoder的每一个层还包含一个全连接的前馈网络，该网络分别且相同地应用于每个位置。该前馈网络包括两个线性变换，并在第一个的最后使用ReLU激活函数。

$$\mathrm{FFN}(x)=\max(0, xW_1 + b_1) W_2 + b_2$$

虽然线性变换在不同位置上是相同的，但它们在层与层之间使用不同的参数。描述这种情况的另一种方式是两个内核大小为1的卷积。输入和输出的维度是$d_{\text{model}}=512$，内层的维度$d_{ff}=2048$。

In [ ]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        d_ff: 隐层的神经元数量。
        """
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # 公式中的max(0, xW+b)其实就是ReLU的公式
        return self.w_2(self.dropout(self.w_1(x).relu()))

## Embeddings and Softmax

与其他的序列转化模型相同，我们也是用可学习的Embeddings来将input和output的token和转换为$d_{\text{model}}$维的向量。

我们也使用常用的线性层和Softmax函数来对decoder的输出做预测，预测下一个token的概率。在我们的模型中，两个Embedding层使用相同的权重。在Embedding层，会乘以权重$\sqrt{d_{\text{model}}}$

关于`nn.Embedding`的使用，可参考[该文章](https://blog.csdn.net/zhaohongfei_358/article/details/122809709)

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        """
        d_model: 词向量的维度
        vocab: 词典的大小。
        """
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

关于为什么Embedding层要乘以权重$\sqrt{d_{\text{model}}}$，可以参考这个[知乎问题](https://www.zhihu.com/question/415263284)中@王四喜和@Quokka的回答。这里我只做一个直观的说明。

由于模型使用了`nn.init.xavier_uniform_`的方式进行初始化，会导致Embedding的结果长这样（`self.lut(x)`的结果）：


```
[[[ 0.0052, -0.0021,  0.0071,  ..., -0.0161, -0.0092, -0.0077],
  [-0.0170,  0.0148, -0.0011,  ...,  0.0118, -0.0128,  0.0143],
  [-0.0137,  0.0146,  0.0196,  ..., -0.0194, -0.0049, -0.0072],
  ...
```

可以看到，都是0.0x开头的。若乘以$\sqrt{d_{\text{model}}}$，结果就变为了：

```
[[[ 0.0593, -0.0239,  0.0808,  ..., -0.1818, -0.1036, -0.0875],
  [-0.1924,  0.1677, -0.0127,  ...,  0.1332, -0.1452,  0.1619],
  [-0.1553,  0.1656,  0.2222,  ..., -0.2198, -0.0554, -0.0815],
  ...
```

所以乘以$\sqrt{d_{\text{model}}}$其实就是起到了一个标准化的作用。


## 位置编码（Position Encoding）

因为Transformer模型不像RNN那样存在循环卷积，为了让模型可以利用输入序列的顺序，我们就必须往序列中注入一些相对位置和绝对位置的信息。因此，我们在input embeddings后增加了“positional encodings”. Positional Encodings与$d_{\text{model}}$有相同的维度，这样他们就可以相加了。

在该工作中，我们使用了不同频率的sine和cosine函数：

$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$

其中$pos$是位置，$i$是维度。也就是说，位置编码的每个维度都对应着一个正弦曲线(sinusoid)。波长形成$2\pi$ 到 $10000\cdot 2\pi$的几何级数。我们选择了这个函数，因为我们假设它允许模型容易地学习相对位置，因为对于任何固定偏移$k$, $PE_{pos+k}$可以表示为$PE_{pos}$的线性函数。

除此之外，我们还在embedding与positional encoding的sum结果上应用了dropout。默认dropout=0.1

上面这段原论文的解释看起来有些晦涩难懂，可以参考[该篇文章](https://zhuanlan.zhihu.com/p/166244505)。

总之就是，我们对第pos个单词的第i个维度，就是加上$PE(pos, 2i)$即可。

In [ ]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # 初始化Shape为(max_len, d_model)的PE (positional encoding)
        pe = torch.zeros(max_len, d_model)
        # 初始化一个tensor [[0, 1, 2, 3, ...]]
        position = torch.arange(0, max_len).unsqueeze(1)
        # 这里就是sin和cos括号中的内容，通过e和ln进行了变换
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        # 计算PE(pos, 2i)
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算PE(pos, 2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 为了方便计算，在最外面在unsqueeze出一个batch
        pe = pe.unsqueeze(0)
        # 如果一个参数不参与梯度下降，但又希望保存model的时候将其保存下来
        # 这个时候就可以用register_buffer
        self.register_buffer("pe", pe)

    def forward(self, x):
        """
        x 为embedding后的inputs，例如(1,7, 128)，batch size为1,7个单词，单词维度为128
        """
        # 将x和positional encoding相加。
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

> 在上面的代码计算`div_term`时，并没有直接使用 $pos / 10000^{2i/d_{\text{model}}}$ 公式，而是使用它的变种 $e^{-2i\ln(10000) / d_{model}}$，我认为原因可能是因为这样计算效率较高。
> 我测试了使用原始公式的代码`1 / torch.pow(10000, torch.arange(0., d_model, 2) / d_model)`，发现结果和变种一样，但速度却慢了一倍。

## 完整模型

这里我们定义一个函数来构建完整模型

In [ ]:
def make_model(
    src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1
):
    """
    Helper: Construct a model from hyperparameters.

    src_vocab：原词典的大小。假设你要做英译汉，那么src_vocab就是英文词典的大小
    tgt_vocab: 目标词典的大小。
    N: EncoderLayer和DecoderLayer的数量
    d_model: 词向量的大小。
    d_ff: 模型中Feed Forward层中隐层神经元的数量
    h: MultiHead中head的数量
    """
    # 用于将模型深度拷贝一份（相当于全新的new一个）
    c = copy.deepcopy
    # 1. 构建多头注意力机制
    attn = MultiHeadedAttention(h, d_model)
    # 2. 构建前馈神经网络
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    # 3. 构建位置编码
    position = PositionalEncoding(d_model, dropout)
    # 4. 构建Transformer模型
    model = EncoderDecoder(
        # Encoder层。
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        # Decoder层，其包含两个Attention层，复制两份
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        # inputs的编码器和位置编码
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        # outputs的编码器和位置编码
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        # Decoder最后的Linear层和Softmax，用于预测下一个token
        Generator(d_model, tgt_vocab),
    )

    # 使用xavier均匀分布的方式进行模型参数初始化
    # 关于为什么要用该初始化方式，我并没有在网上
    # 找到很好的解释。
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

## 推理(Inference)

我们这里使用前向传递来测试一下我们的模型。我们会尝试使用Transformer来记住input。因为模型没训练过，所以输出是随机的。不过在下一章，我们将会构建训练函数来尝试训练我们的模型，让他记住从1到10这几个数字。

In [ ]:
def inference_test():
    """
    测试模型
    """
    # 构架测试模型，原词典和目标词典大小都为11，
    # EncoderLayer和DecoderLayer的数量为2
    test_model = make_model(11, 11, 2)
    test_model.eval()
    # 定义inputs, shape为(1, 10)，即一个句子，该句子10个单词。
    src = torch.LongTensor([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]])
    # 定义src_mask，即所有的词都是有效的，没有填充词
    src_mask = torch.ones(1, 1, 10)

    # 将输入送给encoder，获取memory
    memory = test_model.encode(src, src_mask)

    # 初始化ys为[[0]]，用于保存预测结果，其中0表示'<bos>'
    ys = torch.zeros(1, 1).type_as(src)

    # 循环调用decoder，一个个的进行预测。例如：假设我们要将“I love you”翻译成
    # “我爱你”，则第一次的`ys`为(<bos>)，然后输出为“I”。然后第二次`ys`为(<bos>, I)
    # 输出为"love"，依次类推，直到decoder输出“<eos>”或达到句子长度。
    for i in range(9):
        # 将encoder的输出memory和之前Decoder的所有输出作为参数，让Decoder来预测下一个token
        out = test_model.decode(
            # ys就是Decoder之前的所有输出
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        # 将Decoder的输出送给generator进行预测。这里只取最后一个词的输出进行预测。
        # 因为你传的tgt的词数是变化的，第一次是(<bos>)，第二次是(<bos>, I)
        # 所以你的out的维度也是变化的，变化的就是(batch_size, 词数，词向量)中词数这个维度
        # 既然只能取一个，那当然是最后一个词最合适。
        prob = test_model.generator(out[:, -1])
        # 取出数值最大的那个，它的index在词典中对应的词就是预测结果
        _, next_word = torch.max(prob, dim=1)
        # 取出预测结果
        next_word = next_word.data[0]
        # 将这一次的预测结果和之前的拼到一块，作为之后Decoder的输入
        ys = torch.cat(
            [ys, torch.empty(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    print("Example Untrained Model Prediction:", ys)


def run_tests():
    for _ in range(10):
        inference_test()


run_tests()

# Part 2: 模型训练

本章描述模型的训练机制。

在正式进入训练模型之前，需要先定义一些工具类和方法。

## Batches and Masking

In [ ]:
class Batch:
    """
    定义一个Batch，来存放一个batch的src，tgt，src_mask等对象。
    方便后续的取用
    """

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        """
        src: 和EncoderDecoder#forward中的那个src一致。
             未进行word embedding的句子，例如`[[ 0, 5, 4, 6, 1, 2, 2 ]]`
             上例shape为(1, 7)，即batch size为1，句子大小为7。其中0为bos，
             1为eos, 2为pad

        tgt: 和src类似。是目标句子。
        """
        self.src = src

        """
        构造src_mask：就是将src中pad的部分给盖住，因为这些不属于句子成分，不应该参与计算。
                     例如，src为[[ 0, 5, 4, 6, 1, 2, 2 ]]，则src_mask则为：
                     [[[ True, True, True, True, True, False, False ]]]。因为最后两个2(pad)
                     不属于句子成分。（“<bos>”、“<eos>”和“<unk>”是要算作句子成分的）
        这里unsqueeze一下是因为后续是要对Attention中的scores进行mask，而scores的len(shape)=3,
        为了与scores保持一致，所以unsqueeze(-2)一下。具体可参考attention函数中的注释。
        """
        self.src_mask = (src != pad).unsqueeze(-2)
        if tgt is not None:
            """
            每个句子都去掉最后一个词。例如tgt的Shape为(16, 30)，
            即batch_size为16，每个句子30个单词，执行该代码后为
            (16, 29)。这么做的原因是：tgt存储的是Decoder的输入，
            而Decoder的输入是不可能包含最后一个词的。例如，我们要
            预测`<bos> 我 爱 你 <eos>`，第一次我们会给Decoder
            传`<bos>`，第二次传`<bos> 我`，而最后一次会传
            `<bos> 我 爱 你`。所以tgt不会出现最后一个词，所以要去掉。
            """
            self.tgt = tgt[:, :-1]

            """
            与上面差不多，去掉句子的第一个词，也就是“<bos>”
            tgt_y 存储的是希望预测的结果，所以不需要'<bos>'
            例如，传入encoder的是"<bos> I love you <eos>"，
            初始传入decoder为"<bos>"，则我们想最终能“一个个”
            的预测出“我 爱 你 <eos>”，所以要把<bos>去掉，因为
            其不是我们想要预测的token。
            """
            self.tgt_y = tgt[:, 1:]

            """
            构造tgt_mask：tgt_mask与src_mask略有不同，除了需要盖住pad部分，还需要
                          将对角线右上的也都盖住，具体原因可参考：https://blog.csdn.net/zhaohongfei_358/article/details/125858248
                          例如：[[ 0, 5, 4, 6, 1, 2, 2 ]]，则tgt_mask则为：
                          [[[ True, False, False, False, False, False, False],
                            [ True,  True, False, False, False, False, False],
                            [ True,  True,  True, False, False, False, False],
                            [ True,  True,  True,  True, False, False, False],
                            [ True,  True,  True,  True,  True, False, False],
                            [ True,  True,  True,  True,  True, False, False],
                            [ True,  True,  True,  True,  True, False, False]]]
            """
            self.tgt_mask = self.make_std_mask(self.tgt, pad)

            """
            此Batch的tgt_y的总token数量。值为一个数字，例如 tensor(266) 表示tgt_y有266个有意义的token
            也就是出去<pad>部分的词的数量。保存这个是为了用于loss的正则化，后面loss部分会详细说明。
            注意这里是tgt_y，而tgt_y去掉了“<bos>”，所以token数是不包含“<bos>”的。
            """
            self.ntokens = (self.tgt_y != pad).data.sum()

    @staticmethod
    def make_std_mask(tgt, pad=2):
        """
        生成tgt_mask
        """
        # 首先生成非句子成分部分的mask
        # 例如 [[ 0, 5, 4, 6, 1, 2, 2 ]] 的mask为 [[[ True, True, True, True, True, False, False ]]]
        tgt_mask = (tgt != pad).unsqueeze(-2)

        """
        subsequent_mask用于获取阶梯式Mask。然后再和tgt_mask进行&操作。
        例如：tgt_mask为[[[ True, True, False ]]]
              subsequent_mask结果为：
                        [[[ True, False, False ],
                          [ True, True, False ],
                          [ True, True, True ]]]
              则tgt_mask & subsequent_mask结果为：
                        [[[ True, False, False ],
                          [ True, True, False ],
                          [ True, True, False ]]]
        """
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask

## Training Loop

接下来我们定义一个通用的训练函数和训练状态类：

In [ ]:
class TrainState:
    """用于保存一些训练状态"""

    # step的次数，但注意是一次loss.backward()算一次，或者说一个batch算一次
    # 并不是一次optimizer.step()算一次。在后面的训练代码中，可能会累计多次loss
    # 然后进行一次optimizer.step()
    step: int = 0

    # 参数更新的次数。这个才是optimizer.step()的次数
    accum_step: int = 0

    samples: int = 0  # 记录训练过的样本数量
    tokens: int = 0  # 记录处理过的token数量（target的）

In [ ]:
def run_epoch(
    data_iter,
    model,
    loss_compute,
    optimizer,
    scheduler,
    mode="train",
    accum_iter=1,
    train_state=TrainState(),
):
    """
    进行一个epoch训练

    data_iter: 可迭代对象，一次返回一个Batch对象
    model: Transformer模型，EncoderDecoder类对象
    loss_compute: SimpleLossCompute对象，用于计算损失
    optimizer: Adam优化器。验证时，optimizer是DummyOptimizer
    scheduler：LambdaLR对象，用于调整Adam的学习率，实现WarmUp
               若对调整学习率不熟悉，可参考：https://blog.csdn.net/zhaohongfei_358/article/details/125759911
               验证时，scheduler是DummyScheduler
    accum_iter: 多少个batch更新一次参数，默认为1，也就是每个batch都对参数进行更新
    train_state: TrainState对象，用于保存一些训练状态
    """
    start = time.time()
    # 记录target的总token数，每次打印日志后，进行清0
    tokens = 0
    # 记录tgt_y的总token数，用于对total_loss进行正则化
    total_tokens = 0
    total_loss = 0
    n_accum = 0 # 本次epoch更新了多少次模型参数
    for i, batch in enumerate(data_iter):
        # 前向传递。等价于model(batch.src, batch.tgt, batch.src_mask, batch.tgt_mask)
        # 但注意，这里的out是Decoder的输出，并不是Generator的输出，因为在EncoderDecoder
        # 的forward中并没有使用generator。generator的调用放在了loss_compute中
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )

        """
        计算损失，传入的三个参数分别为：
        1. out: EncoderDecoder的输出，该值并没有过最后的线性层，过线性层被集成在了计算损失中
        2. tgt_y: 要被预测的所有token，例如src为`<bos> I love you <eos>`，则`tgt_y`则为
                  `我 爱 你 <eos>`
        3. ntokens：这批batch中有效token的数量。用于对loss进行正则化。

        返回两个loss，其中loss_node是正则化之后的，所以梯度下降时用这个。
                    而loss是未进行正则化的，用于统计total_loss。
        """
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            # 计算梯度
            loss_node.backward()
            # 记录step次数
            train_state.step += 1
            # 记录样本数量。batch.src.shape[0]获取的是Batch size
            train_state.samples += batch.src.shape[0]
            # 记录处理过的token数
            train_state.tokens += batch.ntokens

            # 如果达到了accum_iter次，就进行一次参数更新
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                # 记录本次epoch的参数更新次数
                n_accum += 1
                # 记录模型的参数更新次数
                train_state.accum_step += 1
            # 更新学习率
            scheduler.step()

        # 累计loss
        total_loss += loss
        # 累计处理过的tokens
        total_tokens += batch.ntokens
        # 累计从上次打印日志开始处理过得tokens
        tokens += batch.ntokens
        # 每40个batch打印一次日志。
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            # 打印一下当前的学习率
            lr = optimizer.param_groups[0]["lr"]
            # 记录这40个batch的消耗时间
            elapsed = time.time() - start
            # 打印日志
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                # i: 本次epoch的第几个batch
                # n_accum: 本次epoch更新了多少次模型参数
                # loss / batch.ntokens: 对loss进行正则化，然后再打印loss，其实这里可以直接用loss_node
                # tokens / elapsed: 每秒可以处理的token数
                # lr: 学习率（learning rate），这里打印学习率的目的是看一下warmup下学习率的变化
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            # 重置开始时间
            start = time.time()
            # 重置token数
            tokens = 0
        del loss
        del loss_node
    # 返回正则化之后的total_loss，返回训练状态
    return total_loss / total_tokens, train_state

## 训练数据和Batch

我们在标准的WMT 2014 English-German数据集上进行了训练，该数据集包含450w个句子对儿。句子使用的是byte-pair编码（subword的一种方法，如果你对subword不了解，可以参考[这篇文章](https://blog.csdn.net/zhaohongfei_358/article/details/123379481)），其中source和target使用的是同一个词典。对于英文到法文的翻译，我们使用了一个巨大的WMT 2014 English-French数据集，其包含3600W个句子，其能拆分出的词典大小为32000。

具有相似长度的句子对儿将会组成一个Batch。每个训练batch都会包含一组句子对儿，其包含大约25000个source tokens和25000个target tokens.

## Optimizer

Transformer使用了Warmup的方式来调整学习率，公式如下：

$$
lrate = d_{\text{model}}^{-0.5} \cdot
  \min({step\_num}^{-0.5},
    {step\_num} \cdot {warmup\_steps}^{-1.5})
$$

等价于：

$$
lrate = \frac{1}{\sqrt{d_{model}}} min(\frac{1}{\sqrt{step\_num}},  step\_num \cdot \frac{1}{warmup\_steps \sqrt{warmup\_steps}})
$$

其中：

- $lrate$: 学习率的调整参数。注意这个并不是学习率，假设你设置的学习率为0.8，则第$i$步的学习率为：$0.8 * lrate(i)$。
- $d_{model}$：模型的维度，即词向量的维度
- $step\_num$: 步数。注意：执行一个backward（也可以说是一个batch）step加1，并不是optimizer.step()一次step加1.
- warmup_steps: warmup多少步。在Transformer中使用的是4000，也就是预热4000步。

In [ ]:
# 学习率调整函数
def rate(step, model_size, factor, warmup):
    # 避免分母为0
    if step == 0:
        step = 1
    # 这里比原公式还多了一个factor，factor默认取1，相当于没有多。
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )

接下来是一个学习率调整的例子，如果你不熟悉自定义学习率，可以参考[这篇文章](https://blog.csdn.net/zhaohongfei_358/article/details/125759911)。

In [ ]:
def example_learning_schedule():
    # 准备3个样例，三个参数分别为：d_model, factor, warmup_steps
    opts = [
        [512, 1, 4000],  # example 1
        [512, 1, 8000],  # example 2
        [256, 1, 4000],  # example 3
    ]

    # 随便定义一个没啥用的模型
    dummy_model = torch.nn.Linear(1, 1)

    # 记录这三个例子的学习率变化
    learning_rates = []

    # 分别运行上面的三个例子
    for idx, example in enumerate(opts):
        # 定义Adam优化器
        optimizer = torch.optim.Adam(
            dummy_model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9
        )
        # 定义LambdaLR调整学习率。"*example"称为序列解包
        lr_scheduler = LambdaLR(
            optimizer=optimizer, lr_lambda=lambda step: rate(step, *example)
        )
        # 用于存储当前样例的学习率变化
        tmp = []
        # 进行20000次训练
        for step in range(20000):
            # 记录学习率
            tmp.append(optimizer.param_groups[0]["lr"])
            # 参数更新
            optimizer.step()
            # 更新学习率。更新学习率要放在optimizer.step()之后
            lr_scheduler.step()
        # 记录当前样例的学习率
        learning_rates.append(tmp)

    # 将学习率变化变成
    learning_rates = torch.tensor(learning_rates)

    # Enable altair to handle more than 5000 rows
    alt.data_transformers.disable_max_rows()

    """
    组织统计图需要的数据，最终效果为：
       |Learning Rate | model_size:warmup | step
    ----------------------------------------------
     0 |     1.74e-07 |          512:4000 |   0
     1 |     1.74e-07 |          512:4000 |   1
     2 |     3.49e-07 |          512:4000 |   2
     ....
    """
    opts_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Learning Rate": learning_rates[warmup_idx, :],
                    "model_size:warmup": ["512:4000", "512:8000", "256:4000"][
                        warmup_idx
                    ],
                    "step": range(20000),
                }
            )
            for warmup_idx in [0, 1, 2]
        ]
    )

    # 绘制统计表
    return (
        alt.Chart(opts_data)
        .mark_line()
        .properties(width=600)
        .encode(x="step", y="Learning Rate", color="model_size:warmup:N")
        .interactive()
    )


example_learning_schedule()

## 正则化

### 标签平滑

在Transformer的训练中，使用Label Smoothing技术对标签做了平滑处理，这样可以减少模型overconfidence，从而减少overfitting。

公式为：

$$
P_{i} = \begin{cases}
   1 ~~~&\text{if } (i=y) \\
   0 ~~~&\text{if } (i\neq y))
\end{cases} ⇒ P_{i} = \begin{cases}
   1-\epsilon ~~~&\text{if } (i=y) \\
   \frac{\epsilon}{K-1}  ~~~&\text{if } (i\neq y))
\end{cases}
$$

其中 $P_i$ 是标签的第i维的值； $K$为类别总数；$\epsilon$为平滑因子，取值范围为$[0,1]$, 通常为0.1，

例如，假设我们的标签是2, 词典大小为5，则对应的one-hot向量则为：$[0, 0, 0, 1, 0]$，我们现在取平滑因子$\epsilon=0.2$，则平滑后的Label为：

$$[0.2/4, 0.2/4, 0.2/4, 1-0.2, 0.2/4]=[0.05, 0.05, 0.05, 0.8, 0.05]$$

之后计算loss时，使用平滑后的标签。直观上的理解就是：模型你别太自信，就算你预测对了也对你进行一点惩罚，防止你过度相信这个特定样本的特定结果。

不过在下面的LabelSmoothing的实现与上面说的略有不同，主要有两点：

1. 这个LabelSmoothing类将损失计算也放了进来，也就是它除了负责标签平滑外，还负责计算损失。
2. 由于词典中有一项是“填充”(`<pad>`)，而网络在什么情况下都不应该预测为`<pad>`，所以在LabelSmoothing的时候不应该让`<pad>`参与。

> 在本代码实现中，所有的`<blank>`指都是`<pad>`

对于第二点，做个详细的说明。

假设我们的词典为`{0: <pad>, 1: <bos>, 2: <eos>, 3: I, 4: love, ..., 1999: you}`，此时词典大小为2000，所以one-hot向量的维度也是2000。如果我们做预测，无论什么情况我们的标签都不应该是`<pad>`（[1, 0, 0, ...]），因为我们不预测`<pad>`。所以我们在平滑的时候，应该忽略`<blank>`，所以公式中的 $K-1$ 也要变成 $K-2$

In [ ]:
class LabelSmoothing(nn.Module):
    """
    实现标签平滑。
    该类除了标签平滑外，还包含了损失的计算。
    所以此类为 LabelSmoothing + LossFunction
    """


    def __init__(self, size, padding_idx, smoothing=0.0):
        """
        size: 目标词典的大小。
        padding_idx: 空格('<blank>')在词典中对应的index，`<blank>`等价于`<pad>`
        smoothing: 平滑因子，0表示不做平滑处理
        """
        super(LabelSmoothing, self).__init__()
        # 定义损失函数，这里使用的是KL Divergence Loss，也是一种多分类常用的损失函数
        # KLDivLoss官方文档：https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        # true distribution，平滑后的标签
        self.true_dist = None

    def forward(self, x, target):
        """
        x: generator输出的概率分布。Shape为(batch_size, 词典大小)
        target: 目标标签。Shape为(batch_size)
        """

        # 确保generator的输出维度和词典大小一致，否则后面计算loss的时候就会出错
        assert x.size(1) == self.size

        # 这句其实是为了创建一个与x有相同Shape的tensor。
        # 这里假设x的shape为(2, 6)，即batch_size为2，词典大小为6。
        true_dist = x.data.clone()

        """
        将true_dist全部填充为 self.smoothing / (self.size - 2)。
        假设 smoothing=0.2，则为全部填充为 0.2 / 4= 0.05
        此时true_dist则为：
        [[0.05, 0.05, 0.05, 0.05, 0.05, 0.05],
         [0.05, 0.05, 0.05, 0.05, 0.05, 0.05]]
        """
        true_dist.fill_(self.smoothing / (self.size - 2))

        """
        scatter_: 官方地址 https://pytorch.org/docs/stable/generated/torch.Tensor.scatter_.html
                  将true_dist的1维度上与target.data.unsqueeze(1)对应的值变为src。
        假设此例中target.data.unsqueeze(1) 为[[2], [3]]，即2个数据的标签分别为2，3
        则true_dist执行过scatter后变为:
        [[0.05, 0.05, 0.8, 0.05, 0.05, 0.05],
         [0.05, 0.05, 0.05, 0.8, 0.05, 0.05]]
        """
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        # 将"空格"所在的index填充为0。
        true_dist[:, self.padding_idx] = 0
        # 找出target中，label为<blank>的标签。例如target为['i', 'love', 'you', '<blank>', '<blank>']
        # 那么mask则为[[1], [3]]，表示第1个和第3个为空格。
        # 但在实际应用场景中标签中不应该出现"<blank>"
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            """
            将"<blank>"所在的label整个设置为0。
            例如，假设现在true_dist为：
            [[0.05, 0.05, 0.8, 0.05, 0.05, 0.05],
             [0.05, 0.05, 0.05, 0.8, 0.05, 0.05],
             [0.8, 0.05, 0.05, 0.05, 0.05, 0.05]]
            其中第3行为“<blank>”的label，则执行完这行代码后，则为：
            [[0.05, 0.05, 0.8, 0.05, 0.05, 0.05],
             [0.05, 0.05, 0.05, 0.8, 0.05, 0.05],
             [0.00, 0.00, 0.00, 0.0, 0.00, 0.00]]
            """
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        # 保存平滑标签后的label
        self.true_dist = true_dist

        """
        使用平滑后的标签计算损失
        由于对`<blank>`部分进行了mask，所以在这部分是不会参与损失计算的
        """
        return self.criterion(x, true_dist.clone().detach())

这里对来看一个Label smoothing的例子：

In [ ]:
def example_label_smoothing():
    # 定义LabelSmoothing, 词典大小为5， <blank>对应index为0，平滑因子为0.4
    crit = LabelSmoothing(5, 0, 0.4)
    # 定义predict，即网络的预测分布。
    # 这里的1e-9在源码中为0。由于KLDivLoss需要对predict求log，而0会导致结果为负无穷
    # 所以我这里将0改成了1e-9。
    predict = torch.FloatTensor(
        [
            [1e-9, 0.2, 0.7, 0.1, 1e-9],
            [1e-9, 0.2, 0.7, 0.1, 1e-9],
            [1e-9, 0.2, 0.7, 0.1, 1e-9],
            [1e-9, 0.2, 0.7, 0.1, 1e-9],
            [1e-9, 0.2, 0.7, 0.1, 1e-9],
        ]
    )
    loss = crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))

    print("loss:", loss)
    print("Before label smoothing:\n", torch.zeros(5, 6).scatter_(1, torch.LongTensor([2, 1, 0, 3, 3]).unsqueeze(1), 1))
    print("After label smoothing:\n", crit.true_dist)

    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect(color="Blue", opacity=1)
        .properties(height=200, width=200)
        .encode(
            alt.X("columns:O", title=None),
            alt.Y("rows:O", title=None),
            alt.Color(
                "target distribution:Q", scale=alt.Scale(scheme="viridis")
            ),
        )
        .interactive()
    )


example_label_smoothing()

> 上图中越黄，表示这个位置的值越接近1, 反之则越接近0。没平滑前，应该是黄色的部分巨黄（=1），其他部位都是巨紫（=0），平滑之后，黄色部位的黄色素给其他紫的部分匀了一些（除了'\<blank\>'位置）。

下面是另一个平滑的例子，从该例子可以看到当模型非常自信的时候就会给予其一个微小的惩罚：

In [ ]:
def loss(x, crit):
    # x是从0到100的一个不断增大的数。 d=x+3，比x大一点。
    d = x + 3
    """
    模拟模型的输出。
    一开始x为1，模型输出为：[[0.0000, 0.2500, 0.2500, 0.2500, 0.2500]]
              此时模型模型还不太会预测
    当x到100时，模型输出为：[[0.0000, 0.9706, 0.0098, 0.0098, 0.0098]]
              此时模型可以很自信的说结果就是 1
    """
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
    # 计算模型损失。由于使用的是KLDivLoss，所以要对predict进行log操作
    return crit(predict.log(), torch.LongTensor([1])).data


def penalization_visualization():
    crit = LabelSmoothing(5, 0, 0.1)
    loss_data = pd.DataFrame(
        {
            # x从1开始不断增大，模拟模型的表现越来越好
            "Loss": [loss(x, crit) for x in range(1, 100)],
            "Steps": list(range(99)),
        }
    ).astype("float")

    return (
        alt.Chart(loss_data)
        .mark_line()
        .properties(width=350)
        .encode(
            x="Steps",
            y="Loss",
        )
        .interactive()
    )


penalization_visualization()

> 从上图可以看出，大约到20次的时候，惩罚效果就出来了。模型虽然很自信的能说出正确答案，但是给他一个小的惩罚，越自信，损失反而越大。

# 第一个例子

我们先从一个最简单的例子开始：从一个很小的词典中随机选取一些词做为输入，目标就是重新输出这些词。我们称为**copy任务**。

## 造数据

In [ ]:
def data_gen(V, batch_size, nbatches):
    """
    生成一组随机数据。（该方法仅用于Demo）
    :param V: 词典的大小
    :param batch_size
    :param nbatches: 生成多少个batch
    :return: yield一个Batch对象
    """

    # 生成{nbatches}个batch
    for i in range(nbatches):
        # 生成一组输入数据
        data = torch.randint(1, V, size=(batch_size, 10))
        # 将每行的第一个词都改为1，即"<bos>"
        data[:, 0] = 1
        # 该数据不需要梯度下降
        src = data.requires_grad_(False).clone().detach()
        tgt = data.requires_grad_(False).clone().detach()
        # 返回一个Batch对象
        yield Batch(src, tgt, 0)

## 损失计算

下面的类可不只是例子要用，最后的实战部分也要用：

In [ ]:
class SimpleLossCompute:
    """
    一个简单的损失计算和训练函数。
    该类除了包含损失计算外，还包含模型generator部分的前向传递。
    如果你对上面这句话不太理解，可参考这篇文章：
    https://blog.csdn.net/zhaohongfei_358/article/details/125759911
    请参考章节：Pytorch 实现梯度下降与参数更新
    """

    def __init__(self, generator, criterion):
        """
        generator: Generator类对象，用于根据Decoder的输出预测下一个token
        criterion: LabelSmoothing类对象，用于对Label进行平滑和计算损失
        """
        self.generator = generator
        self.criterion = criterion

    def __call__(self, x, y, norm):
        """
        x: EncoderDecoder的输出，也就是Decoder的输出
        y: batch.tgt_y，要被预测的所有token，例如src为`<bos> I love you <eos>`，
           则`tgt_y`则为`我 爱 你 <eos>`
        norm: batch.ntokens, tgt_y中的有效token数。用于对loss进行正则化。
        """

        # 调用generator预测token。（EncoderDecoder的forward中并没有调用generator）
        x = self.generator(x)
        """
        这里首先使用KLDivLoss进行了损失计算，随后又除以batch.ntokens对loss进行正则化。
        """
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )

        return sloss.data * norm, sloss

这里详细说明一下上面的正则化，直接通过例子吧。

假设我们的第一个batch（假设batch_size=2）的tgt_y对应的句子为：`<bos> 我 爱 你 <eos> <pad> <pad> ...` 和 `<bos> 苹果 好 吃 <eos> <pad> <pad>`。

则我们的loss相当于对8个预测结果进行了计算，即`我 爱 你 <eos> 苹果 好 吃 <eos>`，也就是$loss = \sum_{i=1}^{8}L(token_i)$

假设我们的第二个batch的tgt_y对应的句子为：`<bos> 中国 是 ...(此处省略100个字) <eos>` 和 `<bos> 华夏 文明 发源 .. <eos>`。

则我们的loss岘港与对200个预测结果进行了计算，也就是$loss = \sum_{i=1}^{200}L(token_i)$

如果不做平均的话，显然第二个大，然后就会出现loss一会大一会小的情况，所以要求一下平均，也就是除以有效的token数。

## 使用贪心算法解码(Greedy Decoding)

简单起见，使用贪心算法解码(Greedy Decoding)进行翻译任务的预测。

在这里，所谓的贪心算法就是Transformer的正常推理过程：先求出encoder的输出memory，然后利用memory一个一个的求出token。

In [ ]:
# 这个代码其实和最开始的inference_test()是一样的
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """
    进行模型推理，推理出所有预测结果。
    :param model: Transformer模型，即EncoderDecoder类对象
    :param src: Encoder的输入inputs，Shape为(batch_size, 词数)
                例如：[[1, 2, 3, 4, 5, 6, 7, 8, 0, 0]]
                即一个句子，该句子有10个词，分别为1,2,...,0
    :param src_mask: src的掩码，掩盖住非句子成分。
    :param max_len: 一个句子的最大长度。
    :param start_symbol: '<bos>' 对应的index，在本例中始终为0
    :return: 预测结果，例如[[1, 2, 3, 4, 5, 6, 7, 8]]
    """

    # 将src送入Transformer的Encoder，输出memory
    memory = model.encode(src, src_mask)
    # 初始化ys为[[0]]，用于保存预测结果，其中0表示'<bos>'
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    # 循环调用decoder，一个个的进行预测。例如：假设我们要将“I love you”翻译成
    # “我爱你”，则第一次的`ys`为(<bos>)，然后输出为“I”。然后第二次`ys`为(<bos>, I)
    # 输出为"love"，依次类推，直到decoder输出“<eos>”或达到句子最大长度。
    for i in range(max_len - 1):
        # 将encoder的输出memory和之前Decoder的所有输出作为参数，让Decoder来预测下一个token
        out = model.decode(
            # ys就是Decoder之前的所有输出
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        # 将Decoder的输出送给generator进行预测。这里只取最后一个词的输出进行预测。
        # 因为你传的tgt的词数是变化的，第一次是(<bos>)，第二次是(<bos>, I)
        # 所以你的out的维度也是变化的，变化的就是(batch_size, 词数，词向量)中词数这个维度
        # 前面的词向量送给generator预测的话，预测出来的也是前面的词，所以只能取最后一个。
        prob = model.generator(out[:, -1])
        # 取出数值最大的那个，它的index在词典中对应的词就是预测结果
        _, next_word = torch.max(prob, dim=1)
        # 取出预测结果
        next_word = next_word.data[0]
        # 将这一次的预测结果和之前的拼到一块，作为之后Decoder的输入
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )

    # 返回最终的预测结果。
    return ys

接下来进行一个简单的模型训练，来实现copy任务。

In [33]:
# 训练一个简单的copy任务
def example_simple_model():
    # 定义词典大小为11
    V = 11

    # 定义损失函数
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)

    # 构建模型，src和tgt的词典大小都为2，Layer数量为2
    model = make_model(V, V, N=2)

    # 使用Adam优化器
    optimizer = torch.optim.Adam(
        model.parameters(), lr=0.5, betas=(0.9, 0.98), eps=1e-9
    )

    # 自定义Warmup学习率
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, model_size=model.src_embed[0].d_model, factor=1.0, warmup=400
        ),
    )

    batch_size = 80
    # 运行20个epoch
    for epoch in range(20):
        # 将模型调整为训练模式
        model.train()
        # 训练一个Batch
        run_epoch(
            # 生成20个batch对象
            data_gen(V, batch_size, 20),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train",
        )
        model.eval()
        # 在一个epoch后，进行模型验证
        run_epoch(
            data_gen(V, batch_size, 5),
            model,
            SimpleLossCompute(model.generator, criterion),
            # 不进行参数更新
            DummyOptimizer(),
            # 不调整学习率
            DummyScheduler(),
            mode="eval",
        )[0]  # run_epoch返回loss和train_state，这里取loss，所以是[0]
        # 但是源码中并没有接收这个loss，所以这个[0]实际上没什么意义

    # 将模型调整为测试模式，准备开始推理
    model.eval()
    # 定义一个src 0-9，看看model能不能重新输出0-9
    src = torch.LongTensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
    # 句子最大长度就是10,
    max_len = src.shape[1]
    # 不需要掩码，因为这10个都是有意义的数字
    src_mask = torch.ones(1, 1, max_len)
    # 使用greedy_decoder函数进行推理
    print(greedy_decode(model, src, src_mask, max_len=max_len, start_symbol=0))


example_simple_model()

Epoch Step:      1 | Accumulation Step:   2 | Loss:   3.15 | Tokens / Sec:   822.1 | Learning Rate: 5.5e-06
Epoch Step:      1 | Accumulation Step:   2 | Loss:   2.05 | Tokens / Sec:   862.9 | Learning Rate: 6.1e-05
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.75 | Tokens / Sec:   988.5 | Learning Rate: 1.2e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.43 | Tokens / Sec:   964.5 | Learning Rate: 1.7e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   1.04 | Tokens / Sec:   975.5 | Learning Rate: 2.3e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.60 | Tokens / Sec:   973.4 | Learning Rate: 2.8e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.36 | Tokens / Sec:   991.6 | Learning Rate: 3.4e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.13 | Tokens / Sec:   979.4 | Learning Rate: 3.9e-04
Epoch Step:      1 | Accumulation Step:   2 | Loss:   0.14 | Tokens / Sec:   990.4 | Learning Rate: 4.5e-04
Epoch Step:      1 | Accumul

如果运行了这个例子，你可能会有一点小疑问：①为什么Epoch Step始终为1；②为什么Accumulation Step始终为2

1. 关于第一点：首先，Epoch Step指的是**在本次epoch中**执行到了第几个batch(从0开始)。其次因为在run_epoch中，会在第1,41,81,...个batch打印日志，而在本例中一共就20个batch，所以一个epoch只会打印一次日志。
2. 第二点：Accumulation Step指的是**在本次epoch中**执行参数更新(`optimizier.step()`)的次数。在第一打印日志时，一共执行了2次参数更新，又因为一个epoch只会打印一次日志，所以该值始终为2。

# Part 3: 实战：德译英

现在我们来进行一个案例实战，我们使用Multi30k German-English 翻译任务。虽然这个任务远小于论文中的WMT任务，但也足以阐明整个系统。

## 数据加载

我们将使用`torchtext`进行数据加载，并使用spacy进行分词。spacy可以参考[这篇文章](https://blog.csdn.net/zhaohongfei_358/article/details/125469155)。

> 加载数据集一定要使用这两个版本`torchdata==0.3.0, torchtext==0.12`，否则会加载失败。

加载分词模型，如果你还没有下载，请使用如下代码进行下载（代码中也会有）：

```
python -m spacy download de_core_news_sm
python -m spacy download en_core_web_sm
```

若在国内使用命令下载失败，请使用离线下载。（注意版本需是3.2.0）[de_core_news_sm下载链接](https://github.com/explosion/spacy-models/releases/tag/de_core_news_sm-3.2.0)，[en_core_web_sm下载链接](https://github.com/explosion/spacy-models/releases/tag/en_core_web_sm-3.2.0)

In [34]:
def load_tokenizers():
    """
    加载spacy分词模型
    :return: 返回德语分词模型和英语分词模型
    """

    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        # 如果报错，说明还未安装分词模型，进行安装后重新加载
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    # 返回德语分词模型和英语分词模型
    return spacy_de, spacy_en

In [35]:
def tokenize(text, tokenizer):
    """
    对text文本进行分词
    :param text: 要分词的文本，例如“I love you”
    :param tokenizer: 分词模型，例如：spacy_en
    :return: 分词结果，例如 ["I", "love", "you"]
    """
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    """
    yield一个token list
    :param data_iter: 包含句子对儿的可迭代对象。例如：
                      [("I love you", "我爱你"), ...]
    :param tokenizer: 分词模型。例如spacy_en
    :param index: 要对句子对儿的哪个语言进行分词，
                  例如0表示对上例的英文进行分词
    :return: yield本轮的分词结果，例如['I', 'love', 'you']
    """
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])

In [36]:
def build_vocabulary(spacy_de, spacy_en):
    """
    构建德语词典和英语词典
    :return: 返回德语词典和英语词典，均为：Vocab对象
             Vocab对象官方地址为：https://pytorch.org/text/stable/vocab.html#vocab
    """
    # 构建德语分词方法
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    # 构建英语分词方法
    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")

    """
    其中train, val, test都是可迭代对象。
    例如：next(iter(train)) 返回一个tuple，为：
    ('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
     'Two young, White males are outside near many bushes.')
    """
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))

    """
    build_vocab_from_iterator：根据一个可迭代对象生成一个词典。
    其返回一个Vocab对象，官方地址为：https://pytorch.org/text/stable/vocab.html#vocab

    其接收是三个参数
    1. iterator，需要传入一个可迭代对象。里面为分好词的数据，例如：
                [("I", "love", "you"), ("you", "love", "me")]
    2. min_freq，最小频率，当一个单词的出现频率达到最小频率后才会被
                 算到词典中。例如，如果min_freq=2，则上例中只有“you”
                 会被算到词典中，因为其他单词都只出现一次。
    3.specials， 特殊词汇，例如'<bos>', '<unk>'等。特殊单词会被加到
                 词典的最前面。

    假设我们调用的是:
    vocab = build_vocab_from_iterator(
        [("I", "love", "you"), ("you", "love", "me")],
        min_freq=1,
        specials=["<s>", "</s>"],
    )
    vocab对应的词典则为：{0:<s>, 1:</s>, 2:love, 3:you, 4:I, 5:me}
    """
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )


    # 开始构建英语词典，与上面一样
    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    # 设置默认index为`<unk>`，后面对于那些不认识的单词就会自动归为`<unk>`
    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    # 返回构建好的德语词典和英语词典
    return vocab_src, vocab_tgt


def load_vocab(spacy_de, spacy_en):
    """
    加载德语词典和英语词典。由于构建词典的过程需要花费一定时间，
    所以该方法就是对build_vocabulary的进一步封装，增加了缓存机制。
    :return: 返回德语词典和英语词典，均为Vocab对象
    """

    # 如果不存在缓存文件，说明是第一次构建词典
    if not exists("vocab.pt"):
        # 构建词典，并写入缓存文件
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        # 如果存在缓存文件，直接加载
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    # 输出些日志：
    print("Finished.\nVocabulary sizes:")
    print("vocab_src size:", len(vocab_src))
    print("vocab_tgt size:", len(vocab_tgt))
    return vocab_src, vocab_tgt


# 全局参数，后续还要用
# 加载德语和英语分词器
spacy_de, spacy_en = load_tokenizers()
# 加载德语词典（源词典）和英语词典（目标词典）
vocab_src, vocab_tgt = load_vocab(spacy_de, spacy_en)

Building German Vocabulary ...
Building English Vocabulary ...
Finished.
Vocabulary sizes:
vocab_src size: 8315
vocab_tgt size: 6384


## Iterators

In [ ]:
def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    """
    Dataloader中的collate_fn函数。该函数的作用是：将文本句子处理成数字句子，然后pad到固定长度，最终batch到一起

    :param batch: 一个batch的语句对。例如：
                  [('Ein Kleinkind ...', 'A toddler in ...'), # [(德语), (英语)
                   ....                                       # ...
                   ...]                                       # ... ]
    :param src_pipeline: 德语分词器，也就是tokenize_de方法，后面会定义
                         其实就是对spacy_de的封装
    :param tgt_pipeline: 英语分词器，也就是tokenize_en方法
    :param src_vocab: 德语词典，Vocab对象
    :param tgt_vocab: 英语词典，Vocab对象
    :param device: cpu或cuda
    :param max_padding: 句子的长度。pad长度不足的句子和裁剪长度过长的句子，
                        目的是让不同长度的句子可以组成一个tensor
    :param pad_id: '<blank>'在词典中对应的index
    :return: src和tgt。处理后并batch后的句子。例如：
             src为：[[0, 4354, 314, ..., 1, 2, 2, ..., 2],  [0, 4905, 8567, ..., 1, 2, 2, ..., 2]]
             其中0是<bos>, 1是<eos>, 2是<blank>
             src的Shape为(batch_size, max_padding)
             tgt同理。
    """

    # 定义'<bos>'的index，在词典中为0，所以这里也是0
    bs_id = torch.tensor([0], device=device)  # <s> token id
    # 定义'<eos>'的index
    eos_id = torch.tensor([1], device=device)  # </s> token id

    # 用于存储处理后的src和tgt
    src_list, tgt_list = [], []
    # 循环遍历句子对儿
    for (_src, _tgt) in batch:
        """
        _src: 德语句子，例如：Ein Junge wirft Blätter in die Luft.
        _tgt: 英语句子，例如：A boy throws leaves into the air.
        """

        """
        将句子进行分词，并将词转成对应的index。例如：
        "I love you" -> ["I", "love", "you"] ->
        [1136, 2468, 1349] -> [0, 1136, 2468, 1349, 1]
        其中0,1是<bos>和<eos>。

        Vocab对象可以将list中的词转为index，例如：
        `vocab_tgt(["I", "love", "you"])` 的输出为：
        [1136, 2468, 1349]
        """
        processed_src = torch.cat(
            # 将<bos>，句子index和<eos>拼到一块
            [
                bs_id,
                torch.tensor(
                    # 进行分词后，转换为index。
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )

        """
        将长度不足的句子进行填充到max_padding的长度的，然后增添到list中

        pad：假设processed_src为[0, 1136, 2468, 1349, 1]
             第二个参数为: (0, 72-5)
             第三个参数为：2
        则pad的意思表示，给processed_src左边填充0个2，右边填充67个2。
        最终结果为：[0, 1136, 2468, 1349, 1, 2, 2, 2, ..., 2]
        """
        src_list.append(
            pad(
                processed_src,
                (0, max_padding - len(processed_src),),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt),),
                value=pad_id,
            )
        )

    # 将多个src句子堆叠到一起
    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)

    # 返回batch后的结果
    return (src, tgt)

In [ ]:
def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128
):
    """
    创建train_dataloader和valid_dataloader
    :param device: cpu或cuda
    :param vocab_src: 源词典，本例中为德语词典
    :param vocab_tgt: 目标词典，本例中为英语词典
    :param spacy_de: 德语分词器
    :param spacy_en: 英语分词器
    :param batch_size: batch_size
    :param max_padding: 句子的最大长度

    :return: train_dataloader和valid_dataloader
    """

    # 定义德语分词器
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    # 定义英语分词器
    def tokenize_en(text):
        return tokenize(text, spacy_en)

    # 创建批处理工具，即应该如何将一批数据汇总成一个Batch
    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    # 加载数据集
    train_iter, valid_iter, test_iter = datasets.Multi30k(
        language_pair=("de", "en")
    )

    """
    将Iterator类型的Dataset转为Map类型的Dataset。如果你不熟悉，可以参考：
    https://blog.csdn.net/zhaohongfei_358/article/details/122742656

    经过测试，发现其实不转也可以。效果没差别
    """
    train_iter_map = to_map_style_dataset(train_iter)
    valid_iter_map = to_map_style_dataset(valid_iter)

    # 构建DataLoader，若DataLoader不熟悉，请参考文章：
    # https://blog.csdn.net/zhaohongfei_358/article/details/122742656
    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader

## 训练模型

In [ ]:
def train_worker(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    config,
    is_distributed=False,
):
    """
    训练模型
    :param device: cpu或cuda
    :param vocab_src: 源词典，本例中为德语词典
    :param vocab_tgt: 目标词典，本例中为英语词典
    :param spacy_de: 德语分词器
    :param spacy_en: 英语分词器
    :param config: 一个保存了配置参数的dict，例如学习率啥的
    """

    print(f"Train worker process using device: {device} for training")

    # 找出目标词典中‘<blank>’所对应的index
    pad_idx = vocab_tgt["<blank>"]
    # 设置词向量大小。
    d_model = 512
    # 构建模型，Layer数为6
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.to(device)

    # 定义损失函数
    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.to(device)

    # 创建train_dataloader和valid_dataloader
    train_dataloader, valid_dataloader = create_dataloaders(
        device,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"],
        max_padding=config["max_padding"]
    )

    # 创建Adam优化器
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )

    # 定义Warmup学习率策略
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )

    # 创建train_state，保存训练状态
    train_state = TrainState()

    # 开始训练
    for epoch in range(config["num_epochs"]):
        model.train()
        print(f"[Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(model.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        """
        展示GPU使用情况，例如：
        | ID | GPU | MEM |
        ------------------
        |  0 | 11% |  6% |
        """
        if torch.cuda.is_available():
            GPUtil.showUtilization()

        # 每训练一个epoch保存一次模型
        file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
        torch.save(model.state_dict(), file_path)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # 在一个epoch后，进行模型验证
        print(f"[Epoch {epoch} Validation ====")
        model.eval()
        # 跑验证集中的数据，看看loss有多少
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(model.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        # 打印验证集的Loss
        print("Validation Loss:", sloss[0].data)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # 全部epoch训练完毕后，保存模型
    file_path = "%sfinal.pt" % config["file_prefix"]
    torch.save(model.state_dict(), file_path)

In [40]:
def load_trained_model():
    """
    加载模型或训练模型。
    若没有找到模型，说明没有训练过，则进行训练
    :return: Transformer对象，即EncoderDecoder类对象
    """

    # 定义一些模型训练参数
    config = {
        "batch_size": 32,
        "num_epochs": 8, # epoch数量
        "accum_iter": 10, # 每10个batch更新一次模型参数
        "base_lr": 1.0,  # 基础学习率，根据这个学习率进行warmup
        "max_padding": 72, # 句子的最大长度
        "warmup": 3000,  # Warmup3000次，也就是从第3000次学习率开始下降
        "file_prefix": "multi30k_model_", # 模型文件前缀名
    }

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model_path = "multi30k_model_final.pt"

    # 如果模型不存在，则训练一个模型
    if not exists(model_path):
        train_worker(device, vocab_src, vocab_tgt, spacy_de, spacy_en, config)

    # 初始化模型实例
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    # 加载模型参数
    model.load_state_dict(torch.load("multi30k_model_final.pt"))
    return model


# 加载或训练模型
model = load_trained_model()

Train worker process using device: cuda for training
[Epoch 0 Training ====
Epoch Step:      1 | Accumulation Step:   1 | Loss:   7.65 | Tokens / Sec:  2701.9 | Learning Rate: 5.4e-07
Epoch Step:     41 | Accumulation Step:   5 | Loss:   7.43 | Tokens / Sec:  2726.4 | Learning Rate: 1.1e-05
Epoch Step:     81 | Accumulation Step:   9 | Loss:   7.01 | Tokens / Sec:  2759.2 | Learning Rate: 2.2e-05
Epoch Step:    121 | Accumulation Step:  13 | Loss:   6.68 | Tokens / Sec:  2730.2 | Learning Rate: 3.3e-05
Epoch Step:    161 | Accumulation Step:  17 | Loss:   6.44 | Tokens / Sec:  2796.8 | Learning Rate: 4.4e-05
Epoch Step:    201 | Accumulation Step:  21 | Loss:   6.29 | Tokens / Sec:  2766.5 | Learning Rate: 5.4e-05
Epoch Step:    241 | Accumulation Step:  25 | Loss:   6.18 | Tokens / Sec:  2768.2 | Learning Rate: 6.5e-05
Epoch Step:    281 | Accumulation Step:  29 | Loss:   5.96 | Tokens / Sec:  2695.9 | Learning Rate: 7.6e-05
Epoch Step:    321 | Accumulation Step:  33 | Loss:   5.82 |

# 测试结果

在最后我们可以使用验证集来简单的测试一下我们的模型


In [ ]:
# Load data and model for output checks
def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results


def run_model_example(n_examples=5):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final.pt", map_location=torch.device("cpu"))
    )

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


run_model_example()

-------完结，撒花--------